In [29]:
from typing import Annotated, Literal
import os
from autogen import ConversableAgent, UserProxyAgent


# Start using Tools

In [30]:
Operator = Literal["+", "-", "*", "/"]


def calculator(a: int, b: int, operator: Annotated[Operator, "operator"]) -> int:
    if operator == "+":
        return a + b
    elif operator == "-":
        return a - b
    elif operator == "*":
        return a * b
    elif operator == "/":
        return a / b
    else:
        raise ValueError("Invalid operator")


# Registering Tools

In [31]:
assistant = ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. "
    "You can help with simple calculations."
    "Return 'TERMINATE' when the task is done.",
    llm_config={
        "config_list": [
            {"model": "gpt-3.5-turbo", "api_key": os.environ["OPENAI_API_KEY"]}
        ]
    },
)

user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None
    and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)
assistant.register_for_llm(
    name="calculator",
    description="A simple calculator",
)(calculator)

user_proxy.register_for_execution(name="calculator")(calculator)


<function __main__.calculator(a: int, b: int, operator: Annotated[Literal['+', '-', '*', '/'], 'operator']) -> int>

# Using the Tool

In [32]:
chat_result = user_proxy.initiate_chat(
    assistant, message="What is (44232 + 13312 / (232 - 32)) * 5?"
)


User (to Assistant):

What is (44232 + 13312 / (232 - 32)) * 5?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_BkvODjUscZzFR3u8J6p5eugP): calculator *****
Arguments: 
{"a":44232,"b":13312,"operator":"/"}
***************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION calculator...
User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_BkvODjUscZzFR3u8J6p5eugP) *****
3.3227163461538463
**********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_YQWueXfsHpQlUyti76u5nzdz): calculator *****
Arguments: 
{"a": 232, "b": 32, "operator": "-"}
************

# Local Model: Ollam + liteLLM ---> llama3

In [ ]:
local_llm_config = {
    "config_list": [
        {
            "model": "NotRequired",
            "api_key": "NotRequired",
            "base_url": "http://0.0.0.0:4000",
        }
    ],
    "cached_seed": None,
}

local_assistant
